#1. 파일 읽기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

filepath = '/content/drive/MyDrive/8.의사결정트리/data/features.txt'
df = pd.read_csv(filepath, sep='\s+', header=None, names=['index', 'name'])
df.head()

,index,name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y


In [3]:
train_file = '/content/drive/MyDrive/8. 의사결정트리/data/train/X_train.txt'
# X_train = pd.read_csv(train_file, sep='\s+', header=None, names=df.name)

> 실행 결과 에러가 발생되는데 이는 feature 정보가 중복되어 발생되는 에러이다.

#2. 중복 feature 변경
> feature 이름이 중복되면 파일을 읽어 들일 경우 에러가 발생되게 된다. 이를 해결하기 위해 feature name에 _1, _2 형식으로 이름을 추가할 것이다.

##2.1 중복 확인
> 현재 데이터 프레임에서 중복된 feature에 대해 확인해 보자

In [ ]:
# name으로 그룹화하여 카운트를 구한다.
df.groupby('name').count()
df_feature_dup = df.groupby('name').count()
# 구한 카운트 정보가 index에 저장되어 이 값이 1보다 큰 값, 즉 중복된 값을 구한다.
df_feature_dup[df_feature_dup['index']>1]

In [5]:
df[df['name'].isin(['fBodyAcc-bandsEnergy()-1,16'])]

,index,name
310,311,"fBodyAcc-bandsEnergy()-1,16"
324,325,"fBodyAcc-bandsEnergy()-1,16"
338,339,"fBodyAcc-bandsEnergy()-1,16"


> 기존의 df에서 name 정보가 'fBodyAcc-bandsEnergy()-1,16'인 값을 확인해 보았더니 위와 같이 3개의 값이 나타나는 것을 볼 수 있다.

In [6]:
dup_list=[310,324,338]

> 추후 중복 확인을 위해 리스트로 목록을 만들었다.

##2.2 중복 변경

In [7]:
df_test = pd.DataFrame([['a'], ['a'], ['a'], ['b'], ['b'], ['c'], ['a'], ['b']], columns=['A'])
df_test

,A
0,a
1,a
2,a
3,b
4,b
5,c
6,a
7,b


In [8]:
df_test['dup_cnt']=df_test.groupby('A').cumcount()
df_test.head(2)

,A,dup_cnt
0,a,0
1,a,1


>cumcount는 각 데이터별 중복된 값을 세는 함수이다.

In [9]:
# df_test.apply(lambda x:x[0]+'_'+str(x[1]), axis=1)
df_test.apply(lambda x:x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)

0      a
1    a_1
2    a_2
3      b
4    b_1
5      c
6    a_3
7    b_2
dtype: object

##2.3 Quiz
> feature의 중복된 값을 변경하시오

In [10]:
df['dup_cnt']=df.groupby('name').cumcount()
df.head(2)

,index,name,dup_cnt
0,1,tBodyAcc-mean()-X,0
1,2,tBodyAcc-mean()-Y,0


> name으로 그룹화 시킨 후 cumcount를 이용하여 중복된 값을 count한 후 dup_cnt에 저장했다.

In [11]:
df_tmp = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
df_tmp[dup_list]

310      fBodyAcc-bandsEnergy()-1,16
324    fBodyAcc-bandsEnergy()-1,16_1
338    fBodyAcc-bandsEnergy()-1,16_2
dtype: object

> x는 데이터 프레임의 한 행으로 x[0]이 index, x[1]은 name, x[2]는 dup_cnt이다. dup_cnt의 값이 0보다 큰 경우가 중복된 값이 되며 이럴 경우 dup_cnt와 _(언더바)를 붙여 중복을 배제할 수 있다.

> 위의 코드에서는 x[2]가 0보가 큰 경우 x[1]에 _와 dup_cnt를 연결하였고 그렇지 않은 경우 x[1]의 값만 출력하였다.

In [12]:
df.loc[dup_list]

,index,name,dup_cnt
310,311,"fBodyAcc-bandsEnergy()-1,16",0
324,325,"fBodyAcc-bandsEnergy()-1,16",1
338,339,"fBodyAcc-bandsEnergy()-1,16",2


> 이전 코드에서 결과를 df_tmp에 저장했으며 원본 데이터인 df에는 적용되지 않은 것을 볼 수 있다.

In [13]:
# df['name'] = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
# df.loc[dup_list]

> 위의 코드를 여러번 실행하면 name에 _숫자 형식으로 데이터가 붙게 된다.

In [14]:
# df['name'] = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
# df.drop('dup_cnt', axis=1, inplace=True)
# df.loc[dup_list]

> 위의 코드는 이전 코드에서 여러 번 실행할 경우 발생했던 문제점을 해결하였다. 하지만 이 코드도 여러 번 실행하면 dup_cnt를 제거하여 x[2] 부분에서 에러가 발생하게 된다.

In [15]:
if len(df.columns)>2:
  df['name'] = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
  df.drop('dup_cnt', axis=1, inplace=True)
df.loc[dup_list]

,index,name
310,311,"fBodyAcc-bandsEnergy()-1,16"
324,325,"fBodyAcc-bandsEnergy()-1,16_1"
338,339,"fBodyAcc-bandsEnergy()-1,16_2"


> 최종적으로 'dup_cnt' feature를 제거 했기때문에 len(df.columns)의 값은 2가 나와 조건문 내부 코드가 실행되지 않는다. 즉, dup_cnt가 존재하면 중복데이터 변경 후 dup_cnt feature를 제거하고 그렇지 않으면 단순하게 출력만 하는 코드이다.

##2.4 백업하기
> 추후 다른 모델에서 적용할 것이기 때문에 현재 데이터 프레임을 파일로 저장하자.

In [16]:
dup_file = '/content/drive/MyDrive/8.의사결정트리/data/features_dup.txt'
df.to_csv(dup_file, header=None)

#3. 데이터 얻어오기
> 훈련데이터와 테스트 데이터를 파일에서 읽어 결과를 출력하세요

In [17]:
file_path = '/content/drive/MyDrive/8.의사결정트리/data/'
X_train = pd.read_csv(file_path+'train/X_train.txt', sep='\s+', header=None, names=df.name)
y_train = pd.read_csv(file_path+'train/y_train.txt', sep='\s+', header=None, names=['action'])

X_test = pd.read_csv(file_path+'test/X_test.txt', sep='\s+', header=None, names=df.name)
y_test = pd.read_csv(file_path+'test/y_test.txt', sep='\s+', header=None, names=['action'])

X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB


In [18]:
y_train['action'].value_counts()

6    1407
5    1374
4    1286
1    1226
2    1073
3     986
Name: action, dtype: int64

#4. 모델 만들기

In [19]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=62)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
pred

array([5, 5, 5, ..., 2, 1, 2])

> 위와 같이 모델을 만든 후 검증데이터로 확인해 보면 예측 결과를 확인할 수 있다.

In [20]:
from sklearn.metrics import accuracy_score

print('정확도 : ', accuracy_score(y_test, pred))
print('하이퍼 파라미터 : ', dt_clf.get_params())

정확도 :  0.8591788259246692
하이퍼 파라미터 :  {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': 62, 'splitter': 'best'}


#5. 튜닝하기

##5.1 한 개의 파라미터 적용

In [21]:
from sklearn.model_selection import GridSearchCV

dt_clf = DecisionTreeClassifier(random_state=62)

def get_best_param(max_depth_list):
  params = {
      'max_depth':max_depth_list
  }
  grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
  grid_cv.fit(X_train, y_train)

  print('Best score : ', grid_cv.best_score_)
  print('Best Hyper Parameter : ', grid_cv.best_params_)

In [22]:
# 10단위 데이터 
# 1 부터 30까지 10단위로 증가. 
# 1, 11, 21, 31
max_depth_list = range(1, 31, 10)
get_best_param(max_depth_list)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   37.9s finished


Best score :  0.8464477402109722
Best Hyper Parameter :  {'max_depth': 21}


> 위의 코드는 1에서 31까지 10단위로 증가하여 1, 11, 21, 31 총 4개의 파라미터로 결과를 확인한 것이다. 실행 결과 11이 최적의 파라미터로 나타났다. 그러면 11 주위의 수가 최적일 가능성이 높다.

In [23]:
# 3단위 데이터 
# 2, 5, 8, 11, 14, 17, 20
max_depth_list = range(2, 20, 3)
get_best_param(max_depth_list)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.3min finished


Best score :  0.8499801606570567
Best Hyper Parameter :  {'max_depth': 8}


In [24]:
# 1단위 데이터 
# 6,7,8,9,10
max_depth_list = range(6, 10, 1)
get_best_param(max_depth_list)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   49.1s finished


Best score :  0.8499801606570567
Best Hyper Parameter :  {'max_depth': 8}


##5.2 두 개의 파라미터 적용

In [25]:
from sklearn.model_selection import GridSearchCV

def get_best_param(max_depth_list, min_samples_split_list):
  params = {
      'max_depth':max_depth_list, 
      'min_samples_split':min_samples_split_list
  }
  grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
  grid_cv.fit(X_train, y_train)

  print('Best score : ', grid_cv.best_score_)
  print('Best Hyper Parameter : ', grid_cv.best_params_)

In [26]:
# 1, 11, 21, 31
# 2, 12, 22, 32
max_depth_list = range(1, 31, 10)
min_samples_split_list = range(2, 32, 10)
get_best_param(max_depth_list, min_samples_split_list)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.9min finished


Best score :  0.8464477402109722
Best Hyper Parameter :  {'max_depth': 21, 'min_samples_split': 2}


> min_samples_split의 최소값은 2임으로 1부터 실행하면 에러가 발생한다.

In [27]:
# 12, 15, 18, 21, 24, 27, 30
# 2, 5, 8, 11
max_depth_list = range(12, 30, 3)
min_samples_split_list = range(2, 11, 3)
get_best_param(max_depth_list, min_samples_split_list)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  5.7min finished


Best score :  0.8482152453095446
Best Hyper Parameter :  {'max_depth': 18, 'min_samples_split': 2}


In [28]:
# 25, 26, 27, 28, 29
# 2
max_depth_list = range(25, 29, 1)
min_samples_split_list = [2]
get_best_param(max_depth_list, min_samples_split_list)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.3min finished


Best score :  0.8464477402109722
Best Hyper Parameter :  {'max_depth': 25, 'min_samples_split': 2}


#6. 하이퍼 파라미터를 적용한 모델

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(max_depth=27, min_samples_split= 2)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)

print('정확도 : ', accuracy_score(y_test, pred))

정확도 :  0.8642687478791992
